In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd


In [9]:
tickers = ['GOOG', 'AAPL', 'MSFT', 'TSLA', 'AMZN']

In [10]:
driver = webdriver.Chrome()
driver.maximize_window()

for ticker in tickers:
    driver.get("https://finance.yahoo.com/")

    driver.find_element(value="yfin-usr-qry").send_keys(ticker)
    driver.find_element(value="header-desktop-search-button").click()
    time.sleep(10)

    driver.find_element('xpath','''//*[@id="quote-nav"]/ul/li[6]/a''').click()
    time.sleep(10)

    driver.find_element('xpath','''//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/div/div/div/span''').click()
    driver.find_element('xpath','''//*[@id="dropdown-menu"]/div/ul[2]/li[4]/button''').click()
    driver.find_element('xpath','''//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/button''').click()

    i = 1
    while(i < 100):
        driver.execute_script(f"window.scrollTo(0, (document.body.scrollHeight)*{i * 10});")
        i = i + 1
        time.sleep(0.5)

    # Pulls the Historical Data
    data = []
    webpage = driver.page_source
    soup = BeautifulSoup(webpage, 'html')
    stock_prices = soup.find_all('tr', class_='BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)')
    for stock_price in stock_prices:
        stock_price = stock_price.find_all('td')
        price = []
        for stock in stock_price:
            price.append(stock.text)
        data.append(price)

    # Generate csv file 
    database = pd.DataFrame(data, columns = ["Date", "Open", "High", "Low", "Close", "Adj. Close", "Volume"])
    database.to_csv(rf'./Dataset/{ticker}.csv', index = False)

driver.quit()
